In [1]:
# @title Import Dependencies

import time
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random
import pyarrow
import requests
from PIL import Image
import os
import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm
from adlfs import AzureBlobFileSystem
import os
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random

import requests
from PIL import Image


In [2]:
# @title Initialize Secrets

os.environ["AZURE_STORAGE_ACCOUNT_NAME"] = "ajdevreddit"

os.environ[
	"AZURE_STORAGE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

os.environ[
	"AZURE_STORE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

In [3]:
# @title File SYstem

class AzureFileStorageAdapter(object):
	def __init__(self, container_name: str = "data"):
		self.__account_name: str = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
		self.__account_key: str = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]
		self.container_name: str = container_name

	def get_file_storage(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(
			account_name=self.__account_name,
			account_key=self.__account_key,
			container_name=self.container_name)

	def get_file_storage_root(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(account_name=self.__account_name, account_key=self.__account_key,
								   container_name=self.container_name)

In [4]:
# @title Get Schema for Curation

tagging_schema = pyarrow.schema(
	[
		pyarrow.field("id", pyarrow.string()),
		pyarrow.field("subreddit", pyarrow.string()),
		pyarrow.field("author", pyarrow.string()),
		pyarrow.field("title", pyarrow.string()),
		pyarrow.field("caption", pyarrow.string()),
		pyarrow.field("hash", pyarrow.string()),
		pyarrow.field("permalink", pyarrow.string()),
		pyarrow.field("original_url", pyarrow.string()),
		pyarrow.field("image_name", pyarrow.string()),
		pyarrow.field("path", pyarrow.string()),
		pyarrow.field("model", pyarrow.string()),
		pyarrow.field("exists", pyarrow.bool_()),
		pyarrow.field("curated", pyarrow.bool_()),
		pyarrow.field("accept", pyarrow.bool_()),
		pyarrow.field("tags", pyarrow.list_(pyarrow.string())),
		pyarrow.field("azure_caption", pyarrow.string()),
		pyarrow.field("thumbnail_path", pyarrow.string()),
		pyarrow.field("thumbnail_exists", pyarrow.bool_()),
		pyarrow.field("thumbnail_curated", pyarrow.bool_()),
		pyarrow.field("thumbnail_accept", pyarrow.bool_()),
		pyarrow.field("additional_captions", pyarrow.list_(pyarrow.string())),
	]
)

In [5]:
# @title Initialize

tqdm.pandas(desc="Progress")

file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

In [6]:
# @title Read in thumbnail_curation

thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow',
										 filesystem=file_system, schema=tagging_schema)

display(thumbnail_curation)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person,human face,clothing,lady,smile,chest,w...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person,human face,skin,eyelash,eyebrow,brown ...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing,person,human face,smile,shoulder,elb...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person,human face,indoor,clothing,woman,chest...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person,human face,smile,clothing,woman,indoor...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20383,av97ze,TrueFMK,xobx321,Bikinis,three women in bikinis posing for a picture wi...,b5ab5a7952ae68b8b3207bc9562e0fb8,/r/TrueFMK/comments/av97ze/bikinis/,https://i.imgur.com/faF0PMP.jpg,av97ze.jpg,data/image/av97ze.jpg,...,True,True,True,[],,,False,False,False,[]
20384,birywt,TrueFMK,NYCplayboy,Barista bombshells,three women in lingerie posing for a picture i...,9c7e904a843d40b93f14856dcb2cdb0d,/r/TrueFMK/comments/birywt/barista_bombshells/,https://i.imgur.com/IxcvEaA.jpg,birywt.jpg,data/image/birywt.jpg,...,True,True,True,[],,,False,False,False,[]
20385,cot1yb,TrueFMK,Bobsagler,College freshmen,three women in bikinis standing next to a swim...,aa0edff56e79f08ee8d887421ac68ceb,/r/TrueFMK/comments/cot1yb/college_freshmen/,https://i.redd.it/cq16x1i5brf31.png,cot1yb.jpg,data/image/cot1yb.jpg,...,True,True,True,[],,,False,False,False,[]
20386,ctkh7j,TrueFMK,NYCplayboy,Sexy schoolgirl servers,three women in short skirts and stockings are ...,1d1ca8f7f70e3a06ba56f4f00dcc842a,/r/TrueFMK/comments/ctkh7j/sexy_schoolgirl_ser...,https://i.imgur.com/e6YgtYy.jpg,ctkh7j.jpg,data/image/ctkh7j.jpg,...,True,True,True,[],,,False,False,False,[]


In [7]:
# @title all_data_from_captions and filtered_data_from_captions

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
											 filesystem=file_system)
all_data_from_captions.set_index('id', inplace=True, drop=False)

filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow',
												  filesystem=file_system)
filtered_data_from_captions.set_index('id', inplace=True, drop=False)

display(all_data_from_captions)
display(filtered_data_from_captions)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
id,,,,,,,,,,
10006iy,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1000cng,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
1000hda,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
1000qpd,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
1000xd6,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
zt4p4w,"{'confidence': 0.4883536100387573, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2007, 'w': 6...",{'content': 'S Copyright @2020 xiuren.com All ...,"{'values': [{'boundingBox': {'h': 2697, 'w': 1...",2023-02-01-preview,"{'height': 2697, 'width': 1800}","{'values': [{'confidence': 0.9860861301422119,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 2305, 'w': 7...",zt4p4w
ztq3co,"{'confidence': 0.6259240508079529, 'text': 'a ...","{'values': [{'boundingBox': {'h': 262, 'w': 26...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1102, 'w': 6...",2023-02-01-preview,"{'height': 1102, 'width': 629}","{'values': [{'confidence': 0.9887521266937256,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1019, 'w': 2...",ztq3co
zwafy3,"{'confidence': 0.5272310376167297, 'text': 'a ...","{'values': [{'boundingBox': {'h': 91, 'w': 99,...","{'content': 'ERSE KAKAKA', 'modelVersion': '20...","{'values': [{'boundingBox': {'h': 918, 'w': 74...",2023-02-01-preview,"{'height': 918, 'width': 740}","{'values': [{'confidence': 0.9973839521408081,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 814, 'w': 56...",zwafy3


,id,captions,dense_captions,meta,tags,smart_crop
id,,,,,,
10006iy,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1000cng,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
1000hda,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
1000qpd,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
1000xd6,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."
...,...,...,...,...,...,...
zt4p4w,zt4p4w,"[{'confidence': 0.4883536100387573, 'text': 'a...","[{'boundingBox': {'h': 2697, 'w': 1800, 'x': 0...","[{'height': 2697, 'width': 1800}]","[{'confidence': 0.9860861301422119, 'name': 'c...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 141..."
ztq3co,ztq3co,"[{'confidence': 0.6259240508079529, 'text': 'a...","[{'boundingBox': {'h': 1102, 'w': 629, 'x': 0,...","[{'height': 1102, 'width': 629}]","[{'confidence': 0.9887521266937256, 'name': 'c...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 516..."
zwafy3,zwafy3,"[{'confidence': 0.5272310376167297, 'text': 'a...","[{'boundingBox': {'h': 918, 'w': 740, 'x': 0, ...","[{'height': 918, 'width': 740}]","[{'confidence': 0.9973839521408081, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 737..."


In [8]:
# @title Get Caption Files

import json

display("=== Obtaining Caption Files List ===")
current_captions = file_system.ls("data/caption")
display(f"current caption files: {len(current_captions)}")

all_data = []
filtered_data = []

for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
	image_id = caption_file.split('/')[-1].split('.')[0]
	if image_id in all_data_from_captions['id'].values or image_id in filtered_data_from_captions['id'].values:
		continue
	try:
		file_size = file_system.size(caption_file)
		if file_size == 0:
			display(f'Empty file -- removing {image_id}', clear=True)
			file_system.rm(caption_file)
			continue
		caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
		caption_data["id"] = image_id
		dense_caption_result = caption_data.get('denseCaptionsResult')
		metadata = caption_data.get('metadata')
		tags_result = caption_data.get('tagsResult')
		smart_crop_result = caption_data.get('smartCropsResult')
		basic_caption = caption_data.get('captionResult')
		_filtered_data = {
			"id": image_id,
			"captions": [basic_caption],
			"dense_captions": dense_caption_result['values'],
			"meta": [metadata],
			"tags": tags_result['values'],
			"smart_crop": smart_crop_result['values']
		}
		all_data.append(caption_data)
		filtered_data.append(_filtered_data)
	except Exception as e:
		display(f" Exception {e} for {image_id}", clear=True)
		continue

'=== Obtaining Caption Files List ==='

'current caption files: 20408'

Reading caption files: 100%|██████████| 20408/20408 [00:32<00:00, 621.57it/s] 


In [9]:
# @title Get new captions

new_all_data_from_captions = pandas.DataFrame(data=all_data)

new_filtered_data_from_captions = pandas.DataFrame(data=filtered_data)

# display(new_all_data_from_captions)
# display(new_filtered_data_from_captions)

combined_new_all_data_from_captions = pandas.concat([all_data_from_captions, new_all_data_from_captions])
combined_new_all_data_from_captions.dropna(axis=1, how='all', inplace=True)
combined_new_all_data_from_captions.reset_index(drop=True, inplace=True)

combined_new_filtered_data_from_captions = pandas.concat([filtered_data_from_captions, new_filtered_data_from_captions])
combined_new_filtered_data_from_captions.dropna(axis=1, how='all', inplace=True)
combined_new_filtered_data_from_captions.reset_index(drop=True, inplace=True)

# display(combined_new_all_data_from_captions)
# display(combined_new_filtered_data_from_captions)

combined_new_all_data_from_captions.to_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
											   filesystem=file_system)

combined_new_filtered_data_from_captions.to_parquet("data/parquet/filtered_data_from_captions.parquet",
													engine='pyarrow', filesystem=file_system)

In [10]:
# @title all_data_from_captions and filtered_data_from_captions

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
											 filesystem=file_system)
display(all_data_from_captions)

	filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow',
												  filesystem=file_system)

display(filtered_data_from_captions)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
2,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
3,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
4,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
20403,"{'confidence': 0.6299172043800354, 'text': 'a ...","{'values': [{'boundingBox': {'h': 125, 'w': 16...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 615, 'w': 75...",2023-02-01-preview,"{'height': 615, 'width': 750}","{'values': [{'confidence': 0.9957964420318604,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 592, 'w': 32...",yshe9g
20404,"{'confidence': 0.5811892747879028, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1032, 'w': 3...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1168, 'w': 1...",2023-02-01-preview,"{'height': 1168, 'width': 1079}","{'values': [{'confidence': 0.9977792501449585,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1061, 'w': 3...",ywtmts
20405,"{'confidence': 0.6229829788208008, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1045, 'w': 2...","{'content': '- - - - - - 1', 'modelVersion': '...","{'values': [{'boundingBox': {'h': 1349, 'w': 1...",2023-02-01-preview,"{'height': 1349, 'width': 1080}","{'values': [{'confidence': 0.9985472559928894,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1094, 'w': 3...",z2pwpm
20406,"{'confidence': 0.547930896282196, 'text': 'a g...","{'values': [{'boundingBox': {'h': 700, 'w': 27...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 960, 'w': 78...",2023-02-01-preview,"{'height': 960, 'width': 786}","{'values': [{'confidence': 0.9916610717773438,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 815, 'w': 28...",zon25k


,id,captions,dense_captions,meta,tags,smart_crop
0,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
2,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
3,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
4,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."
...,...,...,...,...,...,...
20403,yshe9g,"[{'confidence': 0.6299172043800354, 'text': 'a...","[{'boundingBox': {'h': 615, 'w': 750, 'x': 0, ...","[{'height': 615, 'width': 750}]","[{'confidence': 0.9957964420318604, 'name': 'c...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 607..."
20404,ywtmts,"[{'confidence': 0.5811892747879028, 'text': 'a...","[{'boundingBox': {'h': 1168, 'w': 1079, 'x': 0...","[{'height': 1168, 'width': 1079}]","[{'confidence': 0.9977792501449585, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 103..."
20405,z2pwpm,"[{'confidence': 0.6229829788208008, 'text': 'a...","[{'boundingBox': {'h': 1349, 'w': 1080, 'x': 0...","[{'height': 1349, 'width': 1080}]","[{'confidence': 0.9985472559928894, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 106..."
20406,zon25k,"[{'confidence': 0.547930896282196, 'text': 'a ...","[{'boundingBox': {'h': 960, 'w': 786, 'x': 0, ...","[{'height': 960, 'width': 786}]","[{'confidence': 0.9916610717773438, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 783..."


In [11]:
# @title tags

tags = pandas.DataFrame({'id': filtered_data_from_captions.id, 'tags': filtered_data_from_captions.tags})
out = []
for i, r in tqdm(tags.iterrows(), total=len(tags)):
	if r['tags'] is None:
		continue
	tag = pandas.json_normalize(r['tags'])
	tag['id'] = r['id']
	d = tag.to_dict(orient='records')
	out.extend(d)
converted_tags = pandas.DataFrame(data=out)
display(converted_tags)
converted_tags.to_parquet("data/parquet/tags.parquet", engine='pyarrow', filesystem=file_system)

100%|██████████| 20408/20408 [00:16<00:00, 1268.26it/s]


,confidence,name,id
0,0.987604,grass,10006iy
1,0.923687,outdoor,10006iy
2,0.915661,artificial turf,10006iy
3,0.912514,stadium,10006iy
4,0.880675,sports equipment,10006iy
...,...,...,...
308828,0.856076,sandal,zpudb3
308829,0.846750,casual dress,zpudb3
308830,0.841782,pattern (fashion design),zpudb3
308831,0.814569,woman,zpudb3


In [12]:
# @title dense_captions

dense = pandas.DataFrame(
	{'id': filtered_data_from_captions.id, 'dense_captions': filtered_data_from_captions.dense_captions})

out = []
for i, r in tqdm(dense.iterrows(), total=len(dense)):
	if r['dense_captions'] is None:
		continue
	dense_caption = pandas.json_normalize(r['dense_captions'])
	dense_caption['id'] = r['id']
	d = dense_caption.to_dict(orient='records')
	out.extend(d)

converted = pandas.DataFrame(data=out)
display(converted)
converted.to_parquet("data/parquet/dense_captions.parquet", engine='pyarrow', filesystem=file_system)

100%|██████████| 20408/20408 [00:22<00:00, 925.49it/s]


,confidence,text,boundingBox.h,boundingBox.w,boundingBox.x,boundingBox.y,id
0,0.481536,a close-up of a sign,311,1196,1209,1542,10006iy
1,0.485781,a football helmet on grass,1853,7329,0,3068,10006iy
2,0.559779,a lizard on a helmet on a football field,5068,7407,0,0,10006iy
3,0.519275,a lizard on a ball,935,1461,2609,2192,10006iy
4,0.660839,a lizard on a football helmet,2190,2613,2380,2685,10006iy
...,...,...,...,...,...,...,...
169805,0.481301,a woman looking to the side,197,171,320,135,zon25k
169806,0.536570,a close up of a woman's bottom,124,121,339,565,zon25k
169807,0.658436,a woman in a black dress,1191,720,0,0,zpudb3
169808,0.632710,a woman in a black dress,1102,444,105,0,zpudb3


In [13]:
# @title single_caption_data

thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
thumbnail_curation.set_index("id", drop=False, inplace=True)
thumbnail_curation.drop_duplicates(subset="id", inplace=True, keep='first')
thumbnail_curation.dropna(axis=1, how='all', inplace=True)

single_caption_data = pandas.DataFrame(
	{
		'id': all_data_from_captions['id'],
		'azure_caption': [item['text'] for item in all_data_from_captions['captionResult']],
		'tags': [[foo['name'] for foo in item['values']] for item in all_data_from_captions['tagsResult'] if
				 item is not None],
	})

single_caption_data.dropna(axis=1, how='all', inplace=True)
single_caption_data.reset_index(drop=True, inplace=True)
single_caption_data_indexed = single_caption_data.set_index("id", drop=False)

for index, row in tqdm(thumbnail_curation.iterrows(), total=len(thumbnail_curation)):
	thumbnail_curation.at[index, 'azure_caption'] = single_caption_data_indexed.at[index, 'azure_caption']
	thumbnail_curation.at[index, 'tags'] = [",".join(single_caption_data_indexed.at[index, 'tags'])]


display("== Updated With Basic Captions ==")
display(thumbnail_curation)

100%|██████████| 20388/20388 [00:01<00:00, 10740.59it/s]


'== Updated With Basic Captions =='

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person,human face,clothing,lady,smile,chest,w...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person,human face,skin,eyelash,eyebrow,brown ...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing,person,human face,smile,shoulder,elb...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person,human face,indoor,clothing,woman,chest...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person,human face,smile,clothing,woman,indoor...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
av97ze,av97ze,TrueFMK,xobx321,Bikinis,three women in bikinis posing for a picture wi...,b5ab5a7952ae68b8b3207bc9562e0fb8,/r/TrueFMK/comments/av97ze/bikinis/,https://i.imgur.com/faF0PMP.jpg,av97ze.jpg,data/image/av97ze.jpg,...,True,True,True,"[person,human face,clothing,swimwear,undergarm...",a group of women in garments posing for a picture,,False,False,False,[]
birywt,birywt,TrueFMK,NYCplayboy,Barista bombshells,three women in lingerie posing for a picture i...,9c7e904a843d40b93f14856dcb2cdb0d,/r/TrueFMK/comments/birywt/barista_bombshells/,https://i.imgur.com/IxcvEaA.jpg,birywt.jpg,data/image/birywt.jpg,...,True,True,True,"[clothing,person,human face,smile,undergarment...",a group of women posing for a picture,,False,False,False,[]
cot1yb,cot1yb,TrueFMK,Bobsagler,College freshmen,three women in bikinis standing next to a swim...,aa0edff56e79f08ee8d887421ac68ceb,/r/TrueFMK/comments/cot1yb/college_freshmen/,https://i.redd.it/cq16x1i5brf31.png,cot1yb.jpg,data/image/cot1yb.jpg,...,True,True,True,"[clothing,person,human face,undergarment,swimw...",a group of women in garments posing for a picture,,False,False,False,[]


In [14]:
# @title Set Cropping Functions

def get_aspect_ratio(x: object):
	return x['crops'][0]['aspectRatio']


def get_bounding_box(x: object):
	return x['crops'][0]['boundingBox']

In [15]:
# @title Get Croppings

cropping = pandas.DataFrame(
	{'id': filtered_data_from_captions['id'], 'crops': filtered_data_from_captions['smart_crop']}).set_index('id',
																											 drop=False)

cropping['aspectRatio'] = cropping.progress_apply(lambda x: get_aspect_ratio(x), axis=1)
cropping['bounding_box'] = cropping.progress_apply(lambda x: get_bounding_box(x), axis=1)
cropping['x'] = cropping.progress_apply(lambda x: x['bounding_box']['x'], axis=1)
cropping['y'] = cropping.progress_apply(lambda x: x['bounding_box']['y'], axis=1)
cropping['w'] = cropping.progress_apply(lambda x: x['bounding_box']['w'], axis=1)
cropping['h'] = cropping.progress_apply(lambda x: x['bounding_box']['h'], axis=1)

display(cropping)

Progress: 100%|██████████| 20408/20408 [00:00<00:00, 132663.01it/s]


,id,crops,aspectRatio,bounding_box,x,y,w,h
id,,,,,,,,
10006iy,10006iy,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 458...",1.0,"{'h': 4585, 'w': 4586, 'x': 1157, 'y': 475}",1157,475,4586,4585
1000cng,1000cng,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 136...",1.0,"{'h': 1367, 'w': 1367, 'x': 0, 'y': 169}",0,169,1367,1367
1000hda,1000hda,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 280...",1.0,"{'h': 2800, 'w': 2799, 'x': 142, 'y': 550}",142,550,2799,2800
1000qpd,1000qpd,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 740...",1.0,"{'h': 740, 'w': 741, 'x': 173, 'y': 0}",173,0,741,740
1000xd6,1000xd6,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 225...",1.0,"{'h': 2259, 'w': 2259, 'x': 702, 'y': 0}",702,0,2259,2259
...,...,...,...,...,...,...,...,...
yshe9g,yshe9g,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 607...",1.0,"{'h': 607, 'w': 607, 'x': 56, 'y': 0}",56,0,607,607
ywtmts,ywtmts,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 103...",1.0,"{'h': 1037, 'w': 1037, 'x': 0, 'y': 0}",0,0,1037,1037
z2pwpm,z2pwpm,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 106...",1.0,"{'h': 1063, 'w': 1063, 'x': 0, 'y': 0}",0,0,1063,1063


In [16]:
# @title Set function

def create_thumbnail(target_image_id: str, file_names: list, crops: pandas.DataFrame, curated_data: pandas.DataFrame):
	_file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

	out_path = f"data/image/thumbnail/{target_image_id}.jpg"
	try:
		if target_image_id is None or out_path in file_names:
			return out_path

		cropping_information = cropping.loc[crops['id'] == target_image_id]
		if cropping_information is None or len(cropping_information) == 0:
			return "/data/nope"

		record = curated_data.loc[curated_data['id'] == target_image_id]
		record_path = record.to_dict(orient='records')[0]['path']
		image_url = file_system.url(record_path)
		original_image = Image.open(requests.get(image_url, stream=True).raw)
		copied_image = original_image.copy()
		original_image.close()

		cropped = copied_image.crop((cropping_information['x'].values[0],
									 cropping_information['y'].values[0],
									 cropping_information['x'].values[0] +
									 cropping_information['w'].values[0],
									 cropping_information['y'].values[0] +
									 cropping_information['h'].values[0]))
		copied_image.close()

		resized = cropped.resize((512, 512), 1)
		resized.save('temp.jpg')
		resized.close()
		file_system.upload('temp.jpg', out_path, overwrite=True)
		print(f'Thumbnail created for {target_image_id}')
		return out_path

	except Exception as ex:
		print(f'Error creating thumbnail for {target_image_id}: {ex}')
		return "/data/nope"

In [17]:
# # @title Create and load thumbnails

file_names = file_system.ls('data/image/thumbnail')

with tqdm(desc="Creating Thumbnail", total=len(thumbnail_curation)) as pbar:
	for index, row in thumbnail_curation.iterrows():
		if not thumbnail_curation.at[index, 'thumbnail_exists']:
			thumbnail_curation.at[index, 'thumbnail_path'] = create_thumbnail(row['id'], file_names, cropping, thumbnail_curation)
			thumbnail_curation.at[index, 'thumbnail_exists'] = file_system.exists(thumbnail_curation.at[index, 'thumbnail_path'])
			pbar.update(1)
		else:
			pbar.update(1)
			continue

display(thumbnail_curation)

Creating Thumbnail:  97%|█████████▋| 19864/20388 [00:01<00:00, 18018.24it/s]

Thumbnail created for am1jph
Thumbnail created for am5enh
Thumbnail created for amqawo
Thumbnail created for amqeao
Thumbnail created for atea2w
Thumbnail created for b8retz
Thumbnail created for ekqvp3
Thumbnail created for enrrr4
Thumbnail created for exvqb5
Thumbnail created for fqu2ke
Thumbnail created for gc9kfx
Thumbnail created for gkmi1x
Thumbnail created for go2oit
Thumbnail created for gpawzu
Thumbnail created for kz2fpn
Thumbnail created for kz7qwh
Thumbnail created for ldk12u
Thumbnail created for li7xep
Thumbnail created for liiz10
Thumbnail created for logh2w
Thumbnail created for mgeaav
Thumbnail created for mykw8l
Thumbnail created for mzhbk9
Thumbnail created for pi7k2d
Thumbnail created for q992gw
Thumbnail created for qrq7lk
Thumbnail created for xyuzj0
Thumbnail created for eoc305


Creating Thumbnail:  99%|█████████▊| 20085/20388 [00:12<00:00, 18018.24it/s]

Thumbnail created for eodc7r


Creating Thumbnail:  99%|█████████▊| 20086/20388 [00:12<00:00, 395.48it/s]  

...

Creating Thumbnail: 100%|█████████▉| 20359/20388 [01:53<00:04,  6.06it/s]

Thumbnail created for 13lsrxf
Thumbnail created for 13lsx2y
Thumbnail created for 13qjqjg
Thumbnail created for 13sm4r0
Thumbnail created for 13w06mf
Thumbnail created for 140hj2v
Thumbnail created for 141tpt0
Thumbnail created for 1436fuh
Thumbnail created for 144jkyx
Thumbnail created for 145298g
Thumbnail created for 1452m4j
Thumbnail created for 1452st5
Thumbnail created for 145bre2
Thumbnail created for 145bt0z
Thumbnail created for 145btiy
Thumbnail created for 145co4i
Thumbnail created for 145l8gb
Thumbnail created for 145qbx0
Thumbnail created for 146gpxc
Thumbnail created for 146vg8u
Thumbnail created for 146vi6l
Thumbnail created for 146wuh4
Thumbnail created for 146xauo
Thumbnail created for 146xxri
Thumbnail created for as7hoq
Thumbnail created for av97ze
Thumbnail created for birywt
Thumbnail created for cot1yb
Thumbnail created for ctkh7j


Creating Thumbnail: 100%|██████████| 20388/20388 [02:04<00:00, 163.90it/s]

Thumbnail created for cz66c2


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person,human face,clothing,lady,smile,chest,w...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person,human face,skin,eyelash,eyebrow,brown ...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing,person,human face,smile,shoulder,elb...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person,human face,indoor,clothing,woman,chest...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person,human face,smile,clothing,woman,indoor...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
av97ze,av97ze,TrueFMK,xobx321,Bikinis,three women in bikinis posing for a picture wi...,b5ab5a7952ae68b8b3207bc9562e0fb8,/r/TrueFMK/comments/av97ze/bikinis/,https://i.imgur.com/faF0PMP.jpg,av97ze.jpg,data/image/av97ze.jpg,...,True,True,True,"[person,human face,clothing,swimwear,undergarm...",a group of women in garments posing for a picture,data/image/thumbnail/av97ze.jpg,True,False,False,[]
birywt,birywt,TrueFMK,NYCplayboy,Barista bombshells,three women in lingerie posing for a picture i...,9c7e904a843d40b93f14856dcb2cdb0d,/r/TrueFMK/comments/birywt/barista_bombshells/,https://i.imgur.com/IxcvEaA.jpg,birywt.jpg,data/image/birywt.jpg,...,True,True,True,"[clothing,person,human face,smile,undergarment...",a group of women posing for a picture,data/image/thumbnail/birywt.jpg,True,False,False,[]
cot1yb,cot1yb,TrueFMK,Bobsagler,College freshmen,three women in bikinis standing next to a swim...,aa0edff56e79f08ee8d887421ac68ceb,/r/TrueFMK/comments/cot1yb/college_freshmen/,https://i.redd.it/cq16x1i5brf31.png,cot1yb.jpg,data/image/cot1yb.jpg,...,True,True,True,"[clothing,person,human face,undergarment,swimw...",a group of women in garments posing for a picture,data/image/thumbnail/cot1yb.jpg,True,False,False,[]


In [18]:
# @title Copy and Update

thumbnail_curation_copy = thumbnail_curation.copy()

thumbnail_curation_copy.dropna(axis=1, how='all', inplace=True)
thumbnail_curation_copy.drop_duplicates(subset=['id'], inplace=True)
thumbnail_curation_copy.reset_index(drop=True, inplace=True)
display(thumbnail_curation_copy)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person,human face,clothing,lady,smile,chest,w...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person,human face,skin,eyelash,eyebrow,brown ...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing,person,human face,smile,shoulder,elb...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person,human face,indoor,clothing,woman,chest...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person,human face,smile,clothing,woman,indoor...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20383,av97ze,TrueFMK,xobx321,Bikinis,three women in bikinis posing for a picture wi...,b5ab5a7952ae68b8b3207bc9562e0fb8,/r/TrueFMK/comments/av97ze/bikinis/,https://i.imgur.com/faF0PMP.jpg,av97ze.jpg,data/image/av97ze.jpg,...,True,True,True,"[person,human face,clothing,swimwear,undergarm...",a group of women in garments posing for a picture,data/image/thumbnail/av97ze.jpg,True,False,False,[]
20384,birywt,TrueFMK,NYCplayboy,Barista bombshells,three women in lingerie posing for a picture i...,9c7e904a843d40b93f14856dcb2cdb0d,/r/TrueFMK/comments/birywt/barista_bombshells/,https://i.imgur.com/IxcvEaA.jpg,birywt.jpg,data/image/birywt.jpg,...,True,True,True,"[clothing,person,human face,smile,undergarment...",a group of women posing for a picture,data/image/thumbnail/birywt.jpg,True,False,False,[]
20385,cot1yb,TrueFMK,Bobsagler,College freshmen,three women in bikinis standing next to a swim...,aa0edff56e79f08ee8d887421ac68ceb,/r/TrueFMK/comments/cot1yb/college_freshmen/,https://i.redd.it/cq16x1i5brf31.png,cot1yb.jpg,data/image/cot1yb.jpg,...,True,True,True,"[clothing,person,human face,undergarment,swimw...",a group of women in garments posing for a picture,data/image/thumbnail/cot1yb.jpg,True,False,False,[]
20386,ctkh7j,TrueFMK,NYCplayboy,Sexy schoolgirl servers,three women in short skirts and stockings are ...,1d1ca8f7f70e3a06ba56f4f00dcc842a,/r/TrueFMK/comments/ctkh7j/sexy_schoolgirl_ser...,https:

In [19]:
# @title Save new curation data

import datetime
thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
thumbnail_curation.to_parquet(f"data/parquet/{datetime.datetime.now().timestamp()}_thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)

In [20]:
thumbnail_curation_copy.to_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)

In [21]:
thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
display(thumbnail_curation)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person,human face,clothing,lady,smile,chest,w...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person,human face,skin,eyelash,eyebrow,brown ...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing,person,human face,smile,shoulder,elb...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person,human face,indoor,clothing,woman,chest...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person,human face,smile,clothing,woman,indoor...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20383,av97ze,TrueFMK,xobx321,Bikinis,three women in bikinis posing for a picture wi...,b5ab5a7952ae68b8b3207bc9562e0fb8,/r/TrueFMK/comments/av97ze/bikinis/,https://i.imgur.com/faF0PMP.jpg,av97ze.jpg,data/image/av97ze.jpg,...,True,True,True,"[person,human face,clothing,swimwear,undergarm...",a group of women in garments posing for a picture,data/image/thumbnail/av97ze.jpg,True,False,False,[]
20384,birywt,TrueFMK,NYCplayboy,Barista bombshells,three women in lingerie posing for a picture i...,9c7e904a843d40b93f14856dcb2cdb0d,/r/TrueFMK/comments/birywt/barista_bombshells/,https://i.imgur.com/IxcvEaA.jpg,birywt.jpg,data/image/birywt.jpg,...,True,True,True,"[clothing,person,human face,smile,undergarment...",a group of women posing for a picture,data/image/thumbnail/birywt.jpg,True,False,False,[]
20385,cot1yb,TrueFMK,Bobsagler,College freshmen,three women in bikinis standing next to a swim...,aa0edff56e79f08ee8d887421ac68ceb,/r/TrueFMK/comments/cot1yb/college_freshmen/,https://i.redd.it/cq16x1i5brf31.png,cot1yb.jpg,data/image/cot1yb.jpg,...,True,True,True,"[clothing,person,human face,undergarment,swimw...",a group of women in garments posing for a picture,data/image/thumbnail/cot1yb.jpg,True,False,False,[]
20386,ctkh7j,TrueFMK,NYCplayboy,Sexy schoolgirl servers,three women in short skirts and stockings are ...,1d1ca8f7f70e3a06ba56f4f00dcc842a,/r/TrueFMK/comments/ctkh7j/sexy_schoolgirl_ser...,https: